In [ ]:
%pip install snowflake-snowpark-python
%pip install boto3

In [1]:
import json
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc
import snowflake.snowpark
from snowflake.snowpark.types import IntegerType
import boto3
import logging
import sys
import snowflake.snowpark
from snowflake.snowpark.functions import sproc
from snowflake.snowpark.types import IntegerType

In [2]:
# Load snowflake secrets and. create the session
# Snowflake Oauth integration test account
with open("./secrets/GZ45853.json", "r") as f:
    connection_parameters = json.loads(f.read())
test_session = Session.builder.configs(connection_parameters).create()

In [3]:
bucket = "airbyte.alex"
connector = "source-pokeapi"
aws_role_arn = "arn:aws:iam::168714685353:role/snowflake-api-gateway-test"
api_gateway_url = "https://w72cfwmned.execute-api.us-west-1.amazonaws.com/stage"
api_integration_name = f"{connector}_api_integration".replace("-", "_")

In [4]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [5]:
# Create a stage to store the code
create_stage_result = test_session.sql(f"create or REPLACE stage mystage url = 's3://{bucket}'").collect()

INFO:snowflake.connector.cursor:query: [create or REPLACE stage mystage url = 's3://airbyte.alex']
INFO:snowflake.connector.cursor:query execution done


In [6]:
# Create the integration
# WARNING: The gateway's trust relationship must be updated if the api integration is recreated.
def create_api_integration(api_integration_name, aws_role_arn, api_gateway_url):
    return test_session.sql(f"""
    create or replace api integration {api_integration_name}
      api_provider = aws_api_gateway
      api_aws_role_arn = '{aws_role_arn}'
      api_allowed_prefixes = ('{api_gateway_url}')
      enabled = true;
    """).collect()
def describe_api_integration(api_integration_name):
    return test_session.sql(f"describe integration {api_integration_name}").collect()


#create_api_integration(api_integration_name, aws_role_arn, api_gateway_url)
describe_api_integration(api_integration_name)

INFO:snowflake.connector.cursor:query: [describe integration source_pokeapi_api_integration]
INFO:snowflake.connector.cursor:query execution done


[Row(property='ENABLED', property_type='Boolean', property_value='true', property_default='false'),
 Row(property='API_KEY', property_type='String', property_value='', property_default=''),
 Row(property='API_PROVIDER', property_type='String', property_value='AWS_API_GATEWAY', property_default=''),
 Row(property='API_AWS_IAM_USER_ARN', property_type='String', property_value='arn:aws:iam::147018273998:user/6o3f-s-ohss5467', property_default=''),
 Row(property='API_AWS_ROLE_ARN', property_type='String', property_value='arn:aws:iam::168714685353:role/snowflake-api-gateway-test', property_default=''),
 Row(property='API_AWS_EXTERNAL_ID', property_type='String', property_value='GZ45853_SFCRole=2_Q9B1fFjAUub69Ifq6flHOJwBZ/8=', property_default=''),
 Row(property='API_ALLOWED_PREFIXES', property_type='List', property_value='https://w72cfwmned.execute-api.us-west-1.amazonaws.com/stage', property_default='[]'),
 Row(property='API_BLOCKED_PREFIXES', property_type='List', property_value='', prope

In [7]:
# Create external function and translators

request_translator_name = "source_request_translator"
response_translator_name = "source_response_translator"
external_function_name = f"{connector.replace('-', '_')}_external_function"

# Create request_translator
# The same request translator can be used across multiple connectors
def create_request_translator(request_translator_name):
    return test_session.sql(f"""
    create or replace function {request_translator_name}(event object)
    returns object
    language javascript as
    '
    body = EVENT.body.data[0][1]
    suffixUrl = EVENT.body.data[0][2]
    return {{ "body": body, "urlSuffix": suffixUrl}};
    ';
    """).collect()

  
# Create response translator
# The same response translator can be used across multiple connectors
def create_response_translator(response_translator_name):
    return test_session.sql(f"""
    create or replace function {response_translator_name}(event object)
    returns object
    language javascript as
    '
    return {{ "body": {{ "data" : [[0, EVENT]] }}}};
    ';
    """).collect()

# Create external function
# One external function per connector
def create_external_function(external_function_name,
                             api_integration_name,
                             request_translator_name,
                             response_translator_name,
                             api_gateway_url):
    query = f"""
    create or replace external function {external_function_name}(body varchar, urlSuffix varchar)
      returns variant
      api_integration = {api_integration_name}
      request_translator = {request_translator_name}
      response_translator = {response_translator_name}
      as '{api_gateway_url}';
    """
    return test_session.sql(query).collect()

def describe_external_function(external_function_name):
    query = f"describe function {external_function_name} (varchar, varchar)"
    return test_session.sql(query).collect()

print(create_request_translator(request_translator_name))
print(create_response_translator(response_translator_name))
print(create_external_function(external_function_name, api_integration_name, request_translator_name, response_translator_name, api_gateway_url))

describe_external_function(external_function_name)

INFO:snowflake.connector.cursor:query: [create or replace function source_request_translator(event object) returns objec...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Function SOURCE_REQUEST_TRANSLATOR successfully created.')]
INFO:snowflake.connector.cursor:query: [create or replace function source_response_translator(event object) returns obje...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Function SOURCE_RESPONSE_TRANSLATOR successfully created.')]
INFO:snowflake.connector.cursor:query: [create or replace external function source_pokeapi_external_function(body varcha...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Function SOURCE_POKEAPI_EXTERNAL_FUNCTION successfully created.')]
INFO:snowflake.connector.cursor:query: [describe function source_pokeapi_external_function (varchar, varchar)]
INFO:snowflake.connector.cursor:query execution done


[Row(property='signature', value='(BODY VARCHAR, URLSUFFIX VARCHAR)'),
 Row(property='returns', value='VARIANT'),
 Row(property='language', value='EXTERNAL'),
 Row(property='null handling', value='CALLED ON NULL INPUT'),
 Row(property='volatility', value='VOLATILE'),
 Row(property='body', value='https://w72cfwmned.execute-api.us-west-1.amazonaws.com/stage'),
 Row(property='headers', value='null'),
 Row(property='context_headers', value='null'),
 Row(property='max_batch_rows', value='not set'),
 Row(property='request_translator', value='ALEX_TEST.PUBLIC.SOURCE_REQUEST_TRANSLATOR'),
 Row(property='response_translator', value='ALEX_TEST.PUBLIC.SOURCE_RESPONSE_TRANSLATOR'),
 Row(property='compression', value='AUTO')]

In [8]:
# Create a database to be used by the application
def create_database(source_name):
    database_query = f"""
    CREATE DATABASE if not exists {source_name}_app;
    """
    schema_query = f"""
    CREATE SCHEMA if not exists {source_name}_app.app_schema;
    """
    test_session.sql(database_query).collect()
    test_session.sql(schema_query).collect()
create_database("source_pokeapi")

INFO:snowflake.connector.cursor:query: [CREATE DATABASE if not exists source_pokeapi_app;]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE SCHEMA if not exists source_pokeapi_app.app_schema;]
INFO:snowflake.connector.cursor:query execution done


In [9]:
def list_files_to_load(connector):
    s3_paginator = boto3.client('s3').get_paginator('list_objects_v2')

    def keys(bucket_name, prefix='/', delimiter='/', start_after=''):
        prefix = prefix[1:] if prefix.startswith(delimiter) else prefix
        start_after = (start_after or prefix) if prefix.endswith(delimiter) else start_after
        for page in s3_paginator.paginate(Bucket=bucket_name, Prefix=prefix, StartAfter=start_after):
            for content in page.get('Contents', ()):
                key = content['Key']
                yield key

    keys = list(keys(bucket, prefix=f"{connector}"))

    # Compute the list of files to import in the proc
    # For now we're just loading everything
    files_to_load = [f"@mystage/{k}" for k in keys if "pendulum" not in k]
    return files_to_load

In [10]:
# Create a stage to store the code
create_stage_result = test_session.sql(f"create or REPLACE stage mystage url = 's3://{bucket}'").collect()

INFO:snowflake.connector.cursor:query: [create or REPLACE stage mystage url = 's3://airbyte.alex']
INFO:snowflake.connector.cursor:query execution done


In [11]:
# Internal stored procedure that will not be exposed to the consumer

# pendulum has to be installed as a package for reasons...
@sproc(packages=['snowflake-snowpark-python', 'pendulum', 'pandas'], imports=list_files_to_load(connector), name="sync_connector_to_table", replace=True, is_permanent=True, stage_location="@mystage")
def compute(session: snowflake.snowpark.Session, to_table: str, config: dict) -> str:
    from airbyte_cdk.models import ConfiguredAirbyteCatalog, ConfiguredAirbyteStream
    from airbyte_cdk.models import SyncMode, DestinationSyncMode
    import pandas as pd
    
    from source_pokeapi import SourcePokeapi
    logger = logging.getLogger("logger")
    source = SourcePokeapi()

    catalog = source.discover(logger, config)
    configured_catalog = ConfiguredAirbyteCatalog(
        streams=[ConfiguredAirbyteStream(stream=s, sync_mode=SyncMode.full_refresh, destination_sync_mode=DestinationSyncMode.append) for s
                 in catalog.streams])
    
    # hack: get the base url from the first stream...
    base_url = source.streams(config)[0].url_base
        
    def patch_send(session):
        import requests

        def convert_request_to_external_function_input(request: requests.PreparedRequest):
            body = request.body
            headers = request.headers
            url = request.url
            return {"body": body, "headers": headers, "url": url}

        def convert_external_function_output_to_response(output) -> requests.Response:
            response = requests.Response()
            response.status_code = 200
            actual_output = list(output[0].as_dict().items())[0][1]
            
            response_as_json = json.loads(actual_output)
            body = response_as_json["body"]
            
            response._content = json.dumps(body).encode("ascii")
            return response

        def new_session_send(self, request, **kwargs):
            # convert to external function arguments
            args = convert_request_to_external_function_input(request)

            # call external function
            if session:
                #FIXME: No error handling...
                path = "/" + args["url"].replace(base_url, "")
                output_from_external_function = session.sql(f"select source_pokeapi_external_function('{args['body']}', '{path}');").collect()
                response = convert_external_function_output_to_response(output_from_external_function)
            else:
                # This block is just fo testing...
                content = b'{"data": "hello"}'
                response = requests.Response()
                response.status_code = 200
                response._content = content
            return response

        requests.sessions.Session.send = new_session_send
    patch_send(session)
    
    # Filter the columns to avoid running out of memory :(
    keys_to_keep = ["id", "name", "base_experience", "height", "weight"]
    for m in source.read(logger, config, configured_catalog, {}):
        data = m.record.data
        #filtered_data = dict((key, value) for key, value in data.items() if key in keys_to_keep)
        session.sql(f"insert into {to_table} select parse_json('{json.dumps(data)}')").collect()
        #session.create_dataframe([filtered_data]).write.mode('append').save_as_table(to_table)
    return str(data)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:snowflake.connector.cursor:query: [ls '@mystage']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01a763ca-0000-c31d-00...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select package_name, version from information_schema.packages where language='py...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE PROCEDURE sync_connector_to_table(arg1 STRING,arg2 OBJECT) RET...]
INFO:snowflake.connector.cursor:query execution done


In [12]:
# reset output table
test_session.sql("create table if not exists public.test_pokeapi (data variant)").collect()
test_session.sql("delete from public.test_pokeapi;").collect()

INFO:snowflake.connector.cursor:query: [create table if not exists public.test_pokeapi (data variant)]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [delete from public.test_pokeapi;]
INFO:snowflake.connector.cursor:query execution done


[Row(number of rows deleted=0)]

In [13]:
test_session.call("sync_connector_to_table", "public.test_pokeapi", {"pokemon_name": "articuno"})
test_session.call("sync_connector_to_table", "public.test_pokeapi", {"pokemon_name": "zapdos"})
test_session.call("sync_connector_to_table", "public.test_pokeapi", {"pokemon_name": "moltres"})

INFO:snowflake.connector.cursor:query: [CALL sync_connector_to_table('public.test_pokeapi', parse_json('{"pokemon_name":...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CALL sync_connector_to_table('public.test_pokeapi', parse_json('{"pokemon_name":...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CALL sync_connector_to_table('public.test_pokeapi', parse_json('{"pokemon_name":...]
INFO:snowflake.connector.cursor:query execution done


"{'abilities': [{'ability': {'name': 'pressure', 'url': 'https://pokeapi.co/api/v2/ability/46/'}, 'is_hidden': False, 'slot': 1}, {'ability': {'name': 'flame-body', 'url': 'https://pokeapi.co/api/v2/ability/49/'}, 'is_hidden': True, 'slot': 3}], 'base_experience': 290, 'forms': [{'name': 'moltres', 'url': 'https://pokeapi.co/api/v2/pokemon-form/146/'}], 'game_indices': [{'game_index': 73, 'version': {'name': 'red', 'url': 'https://pokeapi.co/api/v2/version/1/'}}, {'game_index': 73, 'version': {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/version/2/'}}, {'game_index': 73, 'version': {'name': 'yellow', 'url': 'https://pokeapi.co/api/v2/version/3/'}}, {'game_index': 146, 'version': {'name': 'gold', 'url': 'https://pokeapi.co/api/v2/version/4/'}}, {'game_index': 146, 'version': {'name': 'silver', 'url': 'https://pokeapi.co/api/v2/version/5/'}}, {'game_index': 146, 'version': {'name': 'crystal', 'url': 'https://pokeapi.co/api/v2/version/6/'}}, {'game_index': 146, 'version': {'name': 'r

In [14]:
test_session.sql("select * from public.test_pokeapi").collect()

INFO:snowflake.connector.cursor:query: [select * from public.test_pokeapi]
INFO:snowflake.connector.cursor:query execution done


[Row(DATA='{\n  "abilities": [\n    {\n      "ability": {\n        "name": "pressure",\n        "url": "https://pokeapi.co/api/v2/ability/46/"\n      },\n      "is_hidden": false,\n      "slot": 1\n    },\n    {\n      "ability": {\n        "name": "snow-cloak",\n        "url": "https://pokeapi.co/api/v2/ability/81/"\n      },\n      "is_hidden": true,\n      "slot": 3\n    }\n  ],\n  "base_experience": 290,\n  "forms": [\n    {\n      "name": "articuno",\n      "url": "https://pokeapi.co/api/v2/pokemon-form/144/"\n    }\n  ],\n  "game_indices": [\n    {\n      "game_index": 74,\n      "version": {\n        "name": "red",\n        "url": "https://pokeapi.co/api/v2/version/1/"\n      }\n    },\n    {\n      "game_index": 74,\n      "version": {\n        "name": "blue",\n        "url": "https://pokeapi.co/api/v2/version/2/"\n      }\n    },\n    {\n      "game_index": 74,\n      "version": {\n        "name": "yellow",\n        "url": "https://pokeapi.co/api/v2/version/3/"\n      }\n    }

In [15]:
test_session.sql("create table if not exists source_pokeapi_app.app_schema.configs (consumer_id varchar, output_table varchar, config variant);").collect()

INFO:snowflake.connector.cursor:query: [create table if not exists source_pokeapi_app.app_schema.configs (consumer_id va...]
INFO:snowflake.connector.cursor:query execution done


[Row(status='CONFIGS already exists, statement succeeded.')]

In [16]:
# Let's insert and read a config to/from a table
def insert_or_update_config(config_table, consumer_id, output_table, config):
    test_session.sql(f"""
    merge into {config_table} a using (select '{consumer_id}' as consumer_id, '{output_table}' as output_table, parse_json('{json.dumps(config)}') as config) as b on a.CONSUMER_ID=b.CONSUMER_ID
      when matched then update set a.config=b.config
      when not matched then insert (consumer_id, output_table, config) values (b.CONSUMER_ID, b.output_table, b.config);
""").collect()
insert_or_update_config("source_pokeapi_app.app_schema.configs", 'id0', "public.test_pokeapi", {"pokemon_name": "abomasnow"})

INFO:snowflake.connector.cursor:query: [merge into source_pokeapi_app.app_schema.configs a using (select 'id0' as consum...]
INFO:snowflake.connector.cursor:query execution done


In [17]:
test_session.sql("select * from source_pokeapi_app.app_schema.configs;").collect()

INFO:snowflake.connector.cursor:query: [select * from source_pokeapi_app.app_schema.configs;]
INFO:snowflake.connector.cursor:query execution done


[Row(CONSUMER_ID='id0', OUTPUT_TABLE='public.test_pokeapi', CONFIG='{\n  "pokemon_name": "abomasnow"\n}')]

In [18]:
# Internal stored procedure that will not be exposed to the consumer

# pendulum has to be installed as a package for reasons...
@sproc(packages=['snowflake-snowpark-python', 'pendulum', 'pandas'], name="sync_consumer_id", replace=True, is_permanent=True, stage_location="@mystage")
def compute_consumer(session: snowflake.snowpark.Session, consumer_id: str) -> str:
    row = session.sql(f"select output_table, config from source_pokeapi_app.app_schema.configs where consumer_id = '{consumer_id}';").collect()[0]
    output_table = row["OUTPUT_TABLE"]
    config = json.loads(row["CONFIG"])
    return session.call("sync_connector_to_table", output_table, config)

INFO:snowflake.connector.cursor:query: [ls '@mystage']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01a763cf-0000-c321-00...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select package_name, version from information_schema.packages where language='py...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE PROCEDURE sync_consumer_id(arg1 STRING) RETURNS STRING LANGUAG...]
INFO:snowflake.connector.cursor:query execution done


In [19]:
test_session.call("sync_consumer_id", "id0")

INFO:snowflake.connector.cursor:query: [CALL sync_consumer_id('id0')]
INFO:snowflake.connector.cursor:query execution done


"{'abilities': [{'ability': {'name': 'snow-warning', 'url': 'https://pokeapi.co/api/v2/ability/117/'}, 'is_hidden': False, 'slot': 1}, {'ability': {'name': 'soundproof', 'url': 'https://pokeapi.co/api/v2/ability/43/'}, 'is_hidden': True, 'slot': 3}], 'base_experience': 173, 'forms': [{'name': 'abomasnow', 'url': 'https://pokeapi.co/api/v2/pokemon-form/460/'}], 'game_indices': [{'game_index': 460, 'version': {'name': 'diamond', 'url': 'https://pokeapi.co/api/v2/version/12/'}}, {'game_index': 460, 'version': {'name': 'pearl', 'url': 'https://pokeapi.co/api/v2/version/13/'}}, {'game_index': 460, 'version': {'name': 'platinum', 'url': 'https://pokeapi.co/api/v2/version/14/'}}, {'game_index': 460, 'version': {'name': 'heartgold', 'url': 'https://pokeapi.co/api/v2/version/15/'}}, {'game_index': 460, 'version': {'name': 'soulsilver', 'url': 'https://pokeapi.co/api/v2/version/16/'}}, {'game_index': 460, 'version': {'name': 'black', 'url': 'https://pokeapi.co/api/v2/version/17/'}}, {'game_index

In [20]:
test_session.sql("select * from public.test_pokeapi").collect()

INFO:snowflake.connector.cursor:query: [select * from public.test_pokeapi]
INFO:snowflake.connector.cursor:query execution done


[Row(DATA='{\n  "abilities": [\n    {\n      "ability": {\n        "name": "pressure",\n        "url": "https://pokeapi.co/api/v2/ability/46/"\n      },\n      "is_hidden": false,\n      "slot": 1\n    },\n    {\n      "ability": {\n        "name": "snow-cloak",\n        "url": "https://pokeapi.co/api/v2/ability/81/"\n      },\n      "is_hidden": true,\n      "slot": 3\n    }\n  ],\n  "base_experience": 290,\n  "forms": [\n    {\n      "name": "articuno",\n      "url": "https://pokeapi.co/api/v2/pokemon-form/144/"\n    }\n  ],\n  "game_indices": [\n    {\n      "game_index": 74,\n      "version": {\n        "name": "red",\n        "url": "https://pokeapi.co/api/v2/version/1/"\n      }\n    },\n    {\n      "game_index": 74,\n      "version": {\n        "name": "blue",\n        "url": "https://pokeapi.co/api/v2/version/2/"\n      }\n    },\n    {\n      "game_index": 74,\n      "version": {\n        "name": "yellow",\n        "url": "https://pokeapi.co/api/v2/version/3/"\n      }\n    }

In [21]:
test_session.sql("delete from source_pokeapi_app.app_schema.configs").collect()

INFO:snowflake.connector.cursor:query: [delete from source_pokeapi_app.app_schema.configs]
INFO:snowflake.connector.cursor:query execution done


[Row(number of rows deleted=1)]

In [22]:
# Procedure that WILL BE exposed to the consumer

# pendulum has to be installed as a package for reasons...
@sproc(packages=['snowflake-snowpark-python', 'pendulum', 'pandas'], name="register_config", replace=True, is_permanent=True, stage_location="@mystage")
def register_config(session: snowflake.snowpark.Session, consumer_id: str, config: dict, output_table: str) -> str:
    def insert_or_update_config(session, config_table, consumer_id, output_table, config):
        session.sql(f"""
        merge into {config_table} a using (select '{consumer_id}' as consumer_id, '{output_table}' as output_table, parse_json('{json.dumps(config)}') as config) as b on a.CONSUMER_ID=b.CONSUMER_ID
          when matched then update set a.config=b.config
          when not matched then insert (consumer_id, output_table, config) values (b.CONSUMER_ID, b.output_table, b.config);
    """).collect()
    insert_or_update_config(session, "source_pokeapi_app.app_schema.configs", consumer_id, output_table, config)
    return "REGISTER SUCCESS"

INFO:snowflake.connector.cursor:query: [ls '@mystage']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01a763d0-0000-c31d-00...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select package_name, version from information_schema.packages where language='py...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE PROCEDURE register_config(arg1 STRING,arg2 OBJECT,arg3 STRING)...]
INFO:snowflake.connector.cursor:query execution done


In [23]:
# Procedule that WILL BE exposed to the consumer

# pendulum has to be installed as a package for reasons...
@sproc(packages=['snowflake-snowpark-python', 'pendulum', 'pandas'], name="schedule_job", replace=True, is_permanent=True, stage_location="@mystage")
def schedule_job(session: snowflake.snowpark.Session, consumer_id: str, warehouse_name: str) -> str:
    create_task_command = f"""
    CREATE OR REPLACE TASK sync warehouse = "{warehouse_name}"
    SCHEDULE = '5 MINUTE'
    as call sync_consumer_id('{consumer_id}')
    """
    session.sql(create_task_command).collect()
    return "REGISTER SUCCESS"

INFO:snowflake.connector.cursor:query: [ls '@mystage']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01a763d0-0000-c31d-00...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select package_name, version from information_schema.packages where language='py...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE PROCEDURE schedule_job(arg1 STRING,arg2 STRING) RETURNS STRING...]
INFO:snowflake.connector.cursor:query execution done


In [24]:
# Creating DB roles for the Snowflake Native Application
database_role = "source_pokeapi_app.shared_db_role"
print(test_session.sql(f"create or replace database role {database_role}").collect())

INFO:snowflake.connector.cursor:query: [create or replace database role source_pokeapi_app.shared_db_role]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Role SOURCE_POKEAPI_APP.SHARED_DB_ROLE successfully created.')]


In [25]:
# Grant usage
print(test_session.sql(f"grant usage on database source_pokeapi_app to database role {database_role};").collect())
print(test_session.sql(f"grant usage on schema source_pokeapi_app.app_schema to database role {database_role};").collect())
print(test_session.sql(f"grant usage on procedure register_config(string, object, string) to database role {database_role}").collect())

INFO:snowflake.connector.cursor:query: [grant usage on database source_pokeapi_app to database role source_pokeapi_app.s...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [grant usage on schema source_pokeapi_app.app_schema to database role source_poke...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [grant usage on procedure register_config(string, object, string) to database rol...]
INFO:snowflake.connector.cursor:query execution done
ERROR:snowflake.snowpark._internal.server_connection:Failed to execute query [queryID: 01a763d5-0000-c321-0000-53b903105496] grant usage on procedure register_config(string, object, string) to database role source_pokeapi_app.shared_db_role
003568 (0A000): 01a763d5-0000-c321-0000-53b903105496: SQL compilation error:
Procedure 'REGISTER_CONFIG(ARG1 VARCHAR, ARG2 OBJE

SnowparkSQLException: (1304): 01a763d5-0000-c321-0000-53b903105496: 003568 (0A000): 01a763d5-0000-c321-0000-53b903105496: SQL compilation error:
Procedure 'REGISTER_CONFIG(ARG1 VARCHAR, ARG2 OBJECT, ARG3 VARCHAR):VARCHAR' does not belong to the database where the database role 'SHARED_DB_ROLE' is created.

In [ ]:
# Create hiddent db role
hidden_db_role = "source_pokeapi_app.hidden_db_role"
print(test_session.sql(f"create or replace database role {hidden_db_role}").collect())
print(test_session.sql(f"grant usage on database source_pokeapi_app to database role {hidden_db_role}").collect())
print(test_session.sql(f"grant usage on schema source_pokeapi_app.app_schema to database role {hidden_db_role};").collect())
print(test_session.sql(f"grant usage on procedure sync_consumer_id(string) to database role {hidden_db_role};").collect())
print(test_session.sql(f"grant usage on procedure sync_connector_to_table(string, object) to database role {hidden_db_role};").collect())

In [ ]:
test_session.call("SOURCE_POKEAPI_APP.APP_SCHEMA.register_config", "id0", {"pokemon_name": "articuno"}, "public.test_pokeapi")

In [ ]:
test_session.sql("describe procedure sync_consumer_id(string)").collect()

In [26]:
test_session.call("schedule_job", "id0", "COMPUTE_WH")

INFO:snowflake.connector.cursor:query: [CALL schedule_job('id0', 'COMPUTE_WH')]
INFO:snowflake.connector.cursor:query execution done


'REGISTER SUCCESS'

In [27]:
test_session.sql("describe task sync").collect()

INFO:snowflake.connector.cursor:query: [describe task sync]
INFO:snowflake.connector.cursor:query execution done


[Row(created_on=datetime.datetime(2022, 10, 3, 15, 45, 34, 634000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), name='SYNC', id='01a763d5-2d55-554a-0000-00000000000e', database_name='ALEX_TEST', schema_name='PUBLIC', owner='ACCOUNTADMIN', comment='', warehouse='COMPUTE_WH', schedule='5 MINUTE', predecessors='[]', state='suspended', definition="call sync_consumer_id('id0')", condition=None, allow_overlapping_execution='false', error_integration='null', last_committed_on=None, last_suspended_on=None)]

In [28]:
test_session.sql("ALTER TASK sync resume;").collect()

INFO:snowflake.connector.cursor:query: [ALTER TASK sync resume;]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Statement executed successfully.')]

In [29]:
test_session.sql("describe task sync").collect()

INFO:snowflake.connector.cursor:query: [describe task sync]
INFO:snowflake.connector.cursor:query execution done


[Row(created_on=datetime.datetime(2022, 10, 3, 15, 45, 34, 634000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), name='SYNC', id='01a763d5-2d55-554a-0000-00000000000e', database_name='ALEX_TEST', schema_name='PUBLIC', owner='ACCOUNTADMIN', comment='', warehouse='COMPUTE_WH', schedule='5 MINUTE', predecessors='[]', state='started', definition="call sync_consumer_id('id0')", condition=None, allow_overlapping_execution='false', error_integration='null', last_committed_on=datetime.datetime(2022, 10, 3, 15, 45, 35, 193000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), last_suspended_on=None)]

In [30]:
test_session.sql("select * from source_pokeapi_app.app_schema.configs").collect()

INFO:snowflake.connector.cursor:query: [select * from source_pokeapi_app.app_schema.configs]
INFO:snowflake.connector.cursor:query execution done


[]

In [37]:
# Creating an installer script
# FIXME: I think the schema should be specific to the connector...
test_session.sql(f"""
CREATE OR REPLACE PROCEDURE source_pokeapi_app.app_schema.installer()
RETURNS STRING
LANGUAGE SQL
EXECUTE AS OWNER
AS $$
  begin
    return 'installer script Done';
  end;
$$;
""").collect()

INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE PROCEDURE source_pokeapi_app.app_schema.installer() RETURNS ST...]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Function INSTALLER successfully created.')]

In [47]:
# Setup a share for the application
test_session.sql(f"""
CREATE SHARE IF NOT EXISTS source_pokeapi_share installer = source_pokeapi_app.app_schema.installer();
""").collect()

INFO:snowflake.connector.cursor:query: [CREATE SHARE IF NOT EXISTS source_pokeapi_share installer = source_pokeapi_app.a...]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Share SOURCE_POKEAPI_SHARE successfully created.')]

In [48]:
# Add objects to the share
print(test_session.sql(f"""grant usage on database source_pokeapi_app to share source_pokeapi_share;""").collect())
print(test_session.sql(f"""grant usage on schema source_pokeapi_app.app_schema to share source_pokeapi_share;""").collect())
print(test_session.sql(f"""grant usage on procedure source_pokeapi_app.app_schema.installer() to share source_pokeapi_share;""").collect())
print(test_session.sql(f"""grant database role source_pokeapi_app.hidden_db_role to share source_pokeapi_share;""").collect())
print(test_session.sql(f"""grant database role source_pokeapi_app.shared_db_role to share source_pokeapi_share;""").collect())

INFO:snowflake.connector.cursor:query: [grant usage on database source_pokeapi_app to share source_pokeapi_share;]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [grant usage on schema source_pokeapi_app.app_schema to share source_pokeapi_shar...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [grant usage on procedure source_pokeapi_app.app_schema.installer() to share sour...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [grant database role source_pokeapi_app.hidden_db_role to share source_pokeapi_sh...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [grant database role source_pokeapi_app.shared_db_role to share

In [49]:
print(test_session.sql(
    f"ALTER share source_pokeapi_share ADD ACCOUNTS = yp50190;"
).collect())

INFO:snowflake.connector.cursor:query: [ALTER share source_pokeapi_share ADD ACCOUNTS = yp50190;]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]


In [50]:
# Open customer session...
with open("./secrets/yp50190.json", "r") as f:
    connection_parameters = json.loads(f.read())
customer_session = Session.builder.configs(connection_parameters).create()

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.9, Platform: macOS-12.2.1-arm64-arm-64bit
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 0.9.0,
"python.version" : 3.8.9,
"python.connector.version" : 2.7.12,
"python.connector.session.id" : 774769150824478,
"os.name" : Darwin



In [52]:
customer_session.sql("create database customer_pokeapi_db from share GZ45853.source_pokeapi_app;").collect()

INFO:snowflake.connector.cursor:query: [create database customer_pokeapi_db from share GZ45853.source_pokeapi_app;]
INFO:snowflake.connector.cursor:query execution done
ERROR:snowflake.snowpark._internal.server_connection:Failed to execute query [queryID: 01a763e0-0000-c3fe-0002-c0a60002c0b2] create database customer_pokeapi_db from share GZ45853.source_pokeapi_app;
002003 (02000): 01a763e0-0000-c3fe-0002-c0a60002c0b2: SQL compilation error:
Share '"GZ45853.SOURCE_POKEAPI_APP"' does not exist or not authorized.


SnowparkSQLException: (1304): 01a763e0-0000-c3fe-0002-c0a60002c0b2: 002003 (02000): 01a763e0-0000-c3fe-0002-c0a60002c0b2: SQL compilation error:
Share '"GZ45853.SOURCE_POKEAPI_APP"' does not exist or not authorized.

In [53]:
# following the tutorial
test_session.sql("CREATE DATABASE time_app;").collect()

INFO:snowflake.connector.cursor:query: [CREATE DATABASE time_app;]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Database TIME_APP successfully created.')]

In [54]:
test_session.sql("CREATE SCHEMA time_app.app_schema").collect()

INFO:snowflake.connector.cursor:query: [CREATE SCHEMA time_app.app_schema]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Schema APP_SCHEMA successfully created.')]

In [55]:
# Create stored procedure
test_session.sql("""
CREATE PROCEDURE time_app.app_schema.record_time()
RETURNS STRING
LANGUAGE SQL
EXECUTE AS OWNER
AS $$
  begin
      INSERT INTO time_schema.time_table VALUES(CURRENT_TIMESTAMP);
      return 'CURRENT TIMESTAMP RECORDED';
  end;

$$;
""").collect()

INFO:snowflake.connector.cursor:query: [CREATE PROCEDURE time_app.app_schema.record_time() RETURNS STRING LANGUAGE SQL E...]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Function RECORD_TIME successfully created.')]

In [56]:
# Create procedure that will be exposed to the consumer
test_session.sql(
"""
CREATE PROCEDURE time_app.app_schema.start_service(wh_name string)

RETURNS STRING
LANGUAGE JAVASCRIPT
EXECUTE AS OWNER
AS $$

var create_task_cmd =
     "CREATE TASK IF NOT EXISTS time_schema.insert_time warehouse = "
        + WH_NAME
        + " SCHEDULE = '5 MINUTE'"
        + " as call app_schema.record_time()";

snowflake.execute({ sqlText: create_task_cmd });

snowflake.execute({ sqlText: `ALTER TASK time_schema.insert_time resume` });

return "SERVICE STARTED";

$$;

""").collect()

INFO:snowflake.connector.cursor:query: [CREATE PROCEDURE time_app.app_schema.start_service(wh_name string)  RETURNS STRI...]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Function START_SERVICE successfully created.')]

In [70]:
# create db roles for the native app
command = """
CREATE or replace DATABASE ROLE time_app.shared_db_role;

GRANT USAGE ON DATABASE time_app TO
DATABASE ROLE time_app.shared_db_role;

GRANT USAGE ON SCHEMA time_app.app_schema TO
DATABASE ROLE time_app.shared_db_role;

GRANT USAGE ON PROCEDURE time_app.app_schema.start_service(string)
TO DATABASE ROLE time_app.shared_db_role;

CREATE or replace DATABASE ROLE time_app.hidden_db_role;

GRANT USAGE ON DATABASE time_app TO
DATABASE ROLE time_app.hidden_db_role;

GRANT USAGE ON SCHEMA time_app.app_schema TO
DATABASE ROLE time_app.hidden_db_role;

GRANT USAGE ON PROCEDURE time_app.app_schema.record_time() TO
DATABASE ROLE time_app.hidden_db_role;"""

for c in command.split(";"):
    if c:
        print(f"command: {c}")
        print(test_session.sql(c).collect())

command: 
CREATE or replace DATABASE ROLE time_app.shared_db_role
INFO:snowflake.connector.cursor:query: [CREATE or replace DATABASE ROLE time_app.shared_db_role]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Role TIME_APP.SHARED_DB_ROLE successfully created.')]
command: 

GRANT USAGE ON DATABASE time_app TO
DATABASE ROLE time_app.shared_db_role
INFO:snowflake.connector.cursor:query: [GRANT USAGE ON DATABASE time_app TO DATABASE ROLE time_app.shared_db_role]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
command: 

GRANT USAGE ON SCHEMA time_app.app_schema TO
DATABASE ROLE time_app.shared_db_role
INFO:snowflake.connector.cursor:query: [GRANT USAGE ON SCHEMA time_app.app_schema TO DATABASE ROLE time_app.shared_db_ro...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
command: 

GRANT USAGE ON PROCEDURE time_app.app_schema.start_service(string)
TO DATABASE ROLE

In [71]:
# Installer script
test_session.sql("""
CREATE PROCEDURE time_app.app_schema.installer()
RETURNS STRING
LANGUAGE SQL
EXECUTE AS OWNER
AS $$
  begin
    CREATE SCHEMA time_schema;
    CREATE TABLE time_schema.time_table(ts timestamp);
    GRANT USAGE ON schema time_schema TO DATABASE ROLE APP_EXPORTER;
    GRANT SELECT ON TABLE time_schema.time_table TO
    DATABASE ROLE APP_EXPORTER;
    GRANT DATABASE ROLE shared_db_role TO DATABASE ROLE APP_EXPORTER;
    return 'installer script Done';
  end;
$$;
""").collect()

INFO:snowflake.connector.cursor:query: [CREATE PROCEDURE time_app.app_schema.installer() RETURNS STRING LANGUAGE SQL EXE...]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Function INSTALLER successfully created.')]

In [72]:
# create share
test_session.sql("CREATE SHARE time_app_share installer = time_app.app_schema.installer();").collect()

INFO:snowflake.connector.cursor:query: [CREATE SHARE time_app_share installer = time_app.app_schema.installer();]
INFO:snowflake.connector.cursor:query execution done


[Row(status='Share TIME_APP_SHARE successfully created.')]

In [73]:
print(test_session.sql("GRANT USAGE ON DATABASE time_app TO SHARE time_app_share;").collect())
print(test_session.sql("GRANT USAGE ON SCHEMA time_app.app_schema TO SHARE time_app_share;").collect())
print(test_session.sql("GRANT USAGE ON PROCEDURE time_app.app_schema.installer() TO SHARE time_app_share;").collect())
print(test_session.sql("GRANT DATABASE ROLE time_app.hidden_db_role TO SHARE time_app_share;").collect())
print(test_session.sql("GRANT DATABASE ROLE time_app.shared_db_role TO SHARE time_app_share;").collect())

INFO:snowflake.connector.cursor:query: [GRANT USAGE ON DATABASE time_app TO SHARE time_app_share;]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [GRANT USAGE ON SCHEMA time_app.app_schema TO SHARE time_app_share;]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [GRANT USAGE ON PROCEDURE time_app.app_schema.installer() TO SHARE time_app_share...]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [GRANT DATABASE ROLE time_app.hidden_db_role TO SHARE time_app_share;]
INFO:snowflake.connector.cursor:query execution done
[Row(status='Statement executed successfully.')]
INFO:snowflake.connector.cursor:query: [GRANT DATABASE ROLE time_app.shared_db_role TO SHARE time_app_share;]
INFO:snowflake.connector.cursor:query ex